## Introduction

This analysis investigates tipping behavior in taxi rides. We wanted to see if the number of passengers in a car affects the average tip percentage given at the end of the ride. By analyzing a large dataset of trips, we compared the average tips for groups of different sizes (specifically, rides with one to six passengers) to identify any clear patterns or differences.

## First implementation

We first load the data using the Parquet parser of Spark.

In [67]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

spark = SparkSession.builder.master('local[2]').getOrCreate()
df = spark.read.format("parquet").option("inferSchema", "true").option("timestampFormat","yyyy-MM-dd HH:mm:ss").option("header", "true").option("mode", "DROPMALFORMED").load("data/*.parquet")
print("Number of trips:",df.count())
df.printSchema()

Number of trips: 179807942
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



In the Spark UI, we can see that loading the data took 0.418 milliseconds.

In [61]:
df.select("PULocationID", "DOLocationID", "tip_amount", "passenger_count", "total_amount") \
    .filter(df["total_amount"] > 0) \
    .withColumn("tip_percent", (F.col("tip_amount") / F.col("total_amount") * 100)) \
    .groupBy("passenger_count") \
    .agg(F.avg("tip_percent").alias("avg_tip_percent")).orderBy(F.desc("avg_tip_percent")) \
    .show()

+---------------+------------------+
|passenger_count|   avg_tip_percent|
+---------------+------------------+
|           NULL|60.063419331989465|
|          112.0|16.666666666666664|
|           96.0|13.043478260869568|
|            1.0| 11.40766087844835|
|            5.0|11.352980006859077|
|            6.0|11.309975849139262|
|            2.0|11.118978531585393|
|            0.0|11.036427380815713|
|            3.0|    10.88123160406|
|            4.0|10.170780710575505|
|            8.0| 9.690808621531673|
|            9.0| 9.566487658538772|
|            7.0| 9.419216760568446|
+---------------+------------------+



Spark UI shows that this operation took 11 seconds.

This first analysis of the tips is not accurate because of incorrect data. Some trips have no passenger number (NULL) or impossible numbers, like 96 or 112. These strange figures made the calculation of the average tip wrong.

## Second implementation

To fix this and get more accurate results, we filter the data to only keep rides with a normal number of passengers, specifically between 1 and 6. This removes the unrealistic trips. Then, we count the total number of rides for each passenger group. We then decided to only keep groups that had more than 1,000 rides. This makes sure our average is based on a large amount of data, not just a few trips.

df.select("PULocationID", "DOLocationID", "tip_amount", "passenger_count", "total_amount") \
    .filter(df["total_amount"] > 0) \
    .withColumn("tip_percent", (F.col("tip_amount") / F.col("total_amount") * 100)) \
    .filter(F.col("passenger_count").between(1, 6)) \
    .groupBy("passenger_count") \
    .agg(F.count("*").alias("nb_courses"),F.avg("tip_percent").alias("avg_tip_percent")) \
    .filter(F.col("nb_courses") > 1000) \
    .orderBy(F.desc("avg_tip_percent")) \
    .show()

Spark UI shows that this operation also took 11 seconds. This shows that the complexity of the operation is far from being the bottleneck of the processing time, instead it is the data size.

## Conclusion

Based on the analysis, the number of passengers has a small but clear effect on tipping habits.

Our results show that single passengers (1.0) gave the highest average tips, at 11.41%. The lowest tips came from groups of four passengers (4.0), who tipped an average of 10.17% of total amount. While the differences are not massive, the data clearly indicates that traveling alone or in a group of four impacts the final tip percentage.

Spark's DataFrame allowed us to handle over 179 million rides for this analysis. We chained commands like .select(), .filter(), and .groupBy() to create a clear and logical "recipe" for our task. This approach is easy to read and efficient, allowing Spark to handle all the difficult work of processing such a huge volume of data.
